In [1]:
#!ipcluster start -n 8 --engines=MPI --profile='mpi' # for parallel run: start the engines using terminal
from ipyparallel import Client
rc = Client(profile='mpi')

In [2]:
%%px
# Import the libraries

import os
import sys
import math
import numpy as np
import pyfftw as ft 
from mpi4py import MPI
import matplotlib
import matplotlib.pyplot as plt
import pyJHTDB
from pyJHTDB.dbinfo import isotropic1024coarse
from pyJHTDB import libJHTDB

from fft3d import FFT3Dfield_new
from EnergySpectrum import EnergySpectrum

In [3]:
%%px

Nx = isotropic1024coarse['nx']; Ny = isotropic1024coarse['ny']; Nz = isotropic1024coarse['nz']
Lx = isotropic1024coarse['lx']; Ly = isotropic1024coarse['ly']; Lz = isotropic1024coarse['lz']

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
nproc = comm.Get_size()
if(rank==0):
    print("n_proc = "+str(nproc))
    print("rank = "+str(rank))

# Computational Domain

nx=Nx//nproc; ny=Ny; nz=Nz
nz_half=nz//2
nek=int(math.sqrt(2.0)/3*Nx)
time = 0.0

chkSz = 32
slabs = nx//chkSz

[stdout:2] 
n_proc = 8
rank = 0


In [4]:
%%px

cacheEnstrophyData = False
loadEnstrophyFromCache = True

folder = "/home/idies/workspace/scratch"
filename = "ref-enstrophy-"+str(rank)+".npz"
file = folder + "/" + filename

if(loadEnstrophyFromCache):
    comm.Barrier(); t1=MPI.Wtime()
    content = np.load(file)
    
    w2 = ft.zeros_aligned((nx,ny,nz), dtype='float32')
    
    if(int(content['nproc'])!=nproc):
        print("Unmatched number of processes. Must first pre-process to adequate number of process")
    w2[:,:,:] = content['w2']
    
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        print("Finished loading")
        sys.stdout.write('Load from disk: {0:.2f} seconds\n'.format(t2-t1))

if(cacheEnstrophyData):
    
    comm.Barrier(); t1=MPI.Wtime()
    np.savez(file,w2=w2,nproc=nproc)
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Caching the data: {0:.2f} seconds\n'.format(t2-t1))

[stdout:2] 
Finished loading
Load from disk: 68.92 seconds


In [5]:
%%px

w2 = 0.5*w2

In [6]:
%%px

avgOmega = np.average(w2)
avgOmegaGl=np.zeros(1,dtype='float32')

comm.Reduce([avgOmega,MPI.REAL],[avgOmegaGl,MPI.REAL],op=MPI.SUM)
avgOmega = avgOmegaGl[0]/nproc
avgOmega = comm.bcast(avgOmega, root=0)

##########################

minw2 = w2.min()
maxw2 = w2.max()

minwGl=np.zeros(nproc,dtype='float32')
maxwGl=np.zeros(nproc,dtype='float32')

comm.Allgather([minw2,MPI.REAL],[minwGl,MPI.REAL])
comm.Allgather([maxw2,MPI.REAL],[maxwGl,MPI.REAL])

minw2 = minwGl.min()
maxw2 = maxwGl.max()

comm.Barrier()

if rank==0:
    print("<w^2>/2 : "+str(avgOmega))
    print("min w2/<w^2> : "+str(minw2/avgOmega))
    print("min w2/<w^2> : "+str(maxw2/avgOmega))

[stdout:2] 
<w^2>/2 : 249.21774292
min w2/<w^2> : 1.43020472082e-07
min w2/<w^2> : 675.170126447


In [7]:
%%px

dx = isotropic1024coarse['dx']
ner = int(1024*np.sqrt(3))
rbins = np.linspace(-0.5*dx,2*np.pi*np.sqrt(3)+0.5*dx,ner+1)

comm.Barrier(); t1=MPI.Wtime()

X = np.zeros((nx,ny,nz), dtype='float32')
Y = np.zeros((nx,ny,nz), dtype='float32')
Z = np.zeros((nx,ny,nz), dtype='float32')
r2 = np.zeros((nx,ny,nz), dtype='float32')

chi = ft.zeros_aligned((nx,ny,nz), dtype='float32')

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Alocating vectors: {0:.2f} seconds\n'.format(t2-t1))

[stdout:2] Alocating vectors: 1.56 seconds


In [8]:
%%px

comm.Barrier(); t1=MPI.Wtime()
for i in range(nx):
    X[i,:,:] = (i+rank*nx)*isotropic1024coarse['dx']
    
for j in range(ny):
    Y[:,j,:] = j*isotropic1024coarse['dy']
    
for k in range(nz):
    Z[:,:,k] = k*isotropic1024coarse['dz']
    
r2[:,:,:] = X[:,:,:]**2+Y[:,:,:]**2+Z[:,:,:]**2

r2rt = np.sqrt(r2)
comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Preparing the real domain for radial integration: {0:.2f} seconds\n'.format(t2-t1))

[stdout:2] Preparing the real domain for radial integration: 170.54 seconds


In [9]:
%%px

BoxL = isotropic1024coarse['lx']
x0 = y0 = z0 = 0.
t = 5

L = BoxL/2.

x1 = x0+isotropic1024coarse['lx']
y1 = y0+isotropic1024coarse['ly']
z1 = z0+isotropic1024coarse['lz']

nx = int((x1-x0)/L)+1
ny = int((y1-y0)/L)+1
nz = int((z1-z0)/L)+1

x1 = x0 + nx*L
y1 = y0 + ny*L 
z1 = z0 + nz*L

In [ ]:
%%px
tl = [1,2,3,4,5,6,7,10,15,20,30,50]
dtl = [1.,0.7,0.5,0.2,0.1,0.05,0.025,0.01,0.005]

lcorr = []
llogr = []

eta = 0.00287

tboxes = []

#t=20
dt = 0.05
#for dt in dtl:
for t in tl:
    comm.Barrier(); t1=MPI.Wtime()
    
    Xs = X[(w2 > t*avgOmega)&(w2 < t*(1+dt)*avgOmega)]
    Ys = Y[(w2 > t*avgOmega)&(w2 < t*(1+dt)*avgOmega)]
    Zs = Z[(w2 > t*avgOmega)&(w2 < t*(1+dt)*avgOmega)]
    
    print(Xs.shape)
    
    hist = np.zeros((Xs.shape[0],3))
    
    hist[:,0] = Xs[:]
    hist[:,1] = Ys[:]
    hist[:,2] = Zs[:]
    
    count = []
    scales = np.logspace(np.log(2*425*eta),np.log(0.1*42.5*eta), num=250, endpoint=True, base=np.e)
    #scales = [0.1*42.5*eta,0.3*42.5*eta]
    
    for L in scales:
        x1 = x0+isotropic1024coarse['lx']
        y1 = y0+isotropic1024coarse['ly']
        z1 = z0+isotropic1024coarse['lz']
        
        nx = int((x1-x0)/L)+1
        ny = int((y1-y0)/L)+1
        nz = int((z1-z0)/L)+1
        
        x1 = x0 + nx*L
        y1 = y0 + ny*L 
        z1 = z0 + nz*L
        
        H, edges = np.histogramdd(hist, bins=(nx,ny,nz), range=((x0,x1),(y0,y1),(z0,z1)), normed=True)
        
        Hglobal = np.zeros(H.shape,dtype='float64')
        comm.Allreduce([H,MPI.DOUBLE],[Hglobal,MPI.DOUBLE],op=MPI.SUM)
        
        Hn = Hglobal[:,:,:]
        Hn[Hn>0] = 1
        numBox = np.sum(Hn)
        gbox = np.zeros(1,dtype='float32')
        gbox[0] = numBox
        gbox = gbox[0]
        
        count.append(gbox)
    
    acount = np.array(count)
    
    tboxes.append(acount[:])
        
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Computing boxcounting numbers: {0:.2f} seconds\n'.format(t2-t1))

In [15]:
%%px 

if rank==0:
    print(tboxes[0])
    print(tboxes[1])
    tfboxes = np.array(tboxes)
    print(tfboxes)
    np.savez('boxcount-enstrophy-shells-dims.npz',tfboxes = tfboxes)

[stdout:2] 
[  2.70000000e+01   2.70000000e+01   2.70000000e+01   2.70000000e+01
   2.70000000e+01   2.70000000e+01   2.70000000e+01   2.70000000e+01
   6.40000000e+01   6.40000000e+01   6.40000000e+01   6.40000000e+01
   6.40000000e+01   6.40000000e+01   6.40000000e+01   6.40000000e+01
   6.40000000e+01   6.40000000e+01   6.40000000e+01   6.40000000e+01
   6.40000000e+01   1.25000000e+02   1.25000000e+02   1.25000000e+02
   1.25000000e+02   1.25000000e+02   1.25000000e+02   1.25000000e+02
   1.25000000e+02   1.25000000e+02   1.25000000e+02   1.25000000e+02
   2.16000000e+02   2.16000000e+02   2.16000000e+02   2.16000000e+02
   2.16000000e+02   2.16000000e+02   2.16000000e+02   2.16000000e+02
   3.43000000e+02   3.43000000e+02   3.43000000e+02   3.43000000e+02
   3.43000000e+02   3.43000000e+02   3.43000000e+02   3.43000000e+02
   5.12000000e+02   5.12000000e+02   5.12000000e+02   5.12000000e+02
   5.12000000e+02   5.12000000e+02   7.29000000e+02   7.29000000e+02
   7.29000000e+02   7.